# Run Layer Fidelity (EPLG100) experiment using qiskit-experiments

`pip install "qiskit-experiments>=0.7" qiskit-ibm-runtime` is required before running this notebook.

In [ ]:
from qiskit_experiments.library.randomized_benchmarking import LayerFidelity

In [ ]:
import numpy as np
import rustworkx as rx
import matplotlib.pyplot as plt

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()

In [ ]:
#specify backend and two-qubit gate to be layered 
backend = service.backend('ibm_kyoto')
twoq_gate = "ecr"
print(f"Device {backend.name} Loaded with {backend.num_qubits} qubits")
print(f"Two Qubit Gate: {twoq_gate}")

### Find the best qubit chain based on reported 2q-gate errors
If you want to benchmark EPLG (error per layered gate) on 100 qubits, set `num_qubits_in_chain==100`

In [ ]:
num_qubits_in_chain = 100
coupling_map = backend.target.build_coupling_map(twoq_gate)
G = coupling_map.graph

In [ ]:
def to_edges(path):
    edges = []
    prev_node = None
    for node in path:
        if prev_node is not None:
            if G.has_edge(prev_node, node):
                edges.append((prev_node, node))
            else:
                edges.append((node, prev_node))
        prev_node = node
    return edges
    
def path_cost(path, correct_by_duration=True):
    """Compute cost of a path. The path is given in node indices form, e.g. [0, 1, 2].
    Use `to_edges` function to obtain edge list, e.g. [(0, 1), (1, 2)]."""
    max_duration = max(backend.target[twoq_gate][qs].duration for qs in to_edges(path))
    
    def gate_error(qpair):
        duration = backend.target[twoq_gate][qpair].duration
        factor = max_duration / duration if correct_by_duration else 1.0
        return factor * backend.target[twoq_gate][qpair].error
    
    gate_cost = -np.prod([1-0.8*gate_error(qs) for qs in to_edges(path)])
    readout_cost = 0
    return gate_cost + readout_cost

def flatten(paths):
    return [val for ps in paths.values() for vals in ps.values() for val in vals]

paths = rx.all_pairs_all_simple_paths(G.to_undirected(multigraph=False), min_depth=num_qubits_in_chain, cutoff=num_qubits_in_chain)
if not flatten(paths):
    raise Exception(f"No qubit chain with length={num_qubits_in_chain} exists in {backend.name}. Try smaller num_qubits_in_chain.")

print(f"Selecting the best from {len(flatten(paths))} candidate paths (will take a few minutes)")
best_qubit_chain =  min(flatten(paths), key=path_cost)
assert(len(best_qubit_chain) == num_qubits_in_chain)
print(f"Predicted LF from reported 2q-gate EPGs: {-path_cost(best_qubit_chain)}")

In [ ]:
np.array(best_qubit_chain)

In [ ]:
# decompose the chain into two disjoint layers
all_pairs = to_edges(best_qubit_chain)
two_disjoint_layers = [all_pairs[0::2], all_pairs[1::2]]
two_disjoint_layers

## Define a LayerFidelity experiment

- `physical_qubits`: Physical qubits to use in this benchmarking
- `two_qubit_layers`: List of list of qubit pairs (a list of qubit pairs corresponds with a two-qubit gate layer), qubit direction matters
- `lengths`: List of layer lengths (the number of depth points)
- `backend`: Backend to be benchmarked
- `num_samples`: How many random circuits should be sampled for each layer length
- `seed`: Seed for the random number generator

[Optional]
- `two_qubit_gate`: Two-qubit gate name to use in layers (If not specified, one of 2q-gates supported in the backend is automatically set.)
- `one_qubit_basis_gates`: One-qubit gate names to use for random 1q-Clifford layers (If not specified, all 1q-gates supported in the backend are automatically set.)

In [ ]:
%%time
lfexp = LayerFidelity(
    physical_qubits=best_qubit_chain,
    two_qubit_layers=two_disjoint_layers,
    lengths=[2, 4, 8, 16, 30, 50, 70, 100, 150, 200, 300, 500],
    backend=backend,
    num_samples=12,
    seed=42,
    # two_qubit_gate="ecr",
    # one_qubit_basis_gates=["rz", "sx", "x"],
)

In [ ]:
lfexp.experiment_options

In [ ]:
print(f"Two Qubit Gate: {lfexp.experiment_options.two_qubit_gate}")
print(f"One Qubit Gate Set: {lfexp.experiment_options.one_qubit_basis_gates}")

### Quick look at a RB circuit
`circuits_geenrator` is useful for that.

In [ ]:
%%time
# look at one of the first three 2Q direct RB circuits quickly
circ_iter = lfexp.circuits_generator()
first_three_circuits = list(next(circ_iter) for _ in range(3))
first_three_circuits[1].draw(output="mpl", style="clifford", idle_wires=False, fold=-1)

### Generating all RB circuits takes a bit more time

In [ ]:
%%time
# generate all circuits to run
circuits = lfexp.circuits()
print(f"{len(circuits)} circuits are generated.")

## Run the Layer Fidelity experiment

Run = Generate circuits and submit the job to the backend.

Set `exp_data.auto_save = True` to store the experiment data using qiskit-ibm-experiment service.

In [ ]:
%%time
#number of shots per job
nshots = 400

# Run the LF experiment (generate circuits and submit the job)
exp_data = lfexp.run(shots=nshots)
# exp_data.auto_save = True
print(f"Run experiment: ID={exp_data.experiment_id} with jobs {exp_data.job_ids}]")

## Wait for the result
Retrieve the result after the job is completed and run analysis (fit RB curves and estimate process fidelity for each 1Q/2Q direct RB)

In [ ]:
%%time
exp_data.block_for_results()

### Print out the result data as a table

In [ ]:
# Get the result data as a pandas DataFrame
df = exp_data.analysis_results(dataframe=True)

In [ ]:
# LF and LF of each disjoing set
df[(df.name == "LF") | (df.name == "EPLG") | (df.name == "SingleLF")]

In [ ]:
# 1Q/2Q process fidelities
df[(df.name == "ProcessFidelity")].head()

### Display plots (1Q/2Q simultaneous direct RB curves)

In [ ]:
# Plot 1Q/2Q simultaneous direct RB curves
for i in range(0, 5):
    display(exp_data.figure(i))

### Filter qubit pairs with bad process fidelity 

In [ ]:
pfdf = df[(df.name == "ProcessFidelity")]
pfdf[pfdf.value < 0.8]

### Invesitigate analysis results labeled "bad" quality

In [ ]:
# find bad quality analysis results
pfdf[pfdf.quality=="bad"]

In [ ]:
RUN_TIME = df[(df.name == "LF")].run_time.values[0]
RUN_TIME

## Plot Layer Fidelity (EPLG) by chain length

In [ ]:
# fill Process Fidelity values with zeros
pfdf = pfdf.fillna({"value": 0})

In [ ]:
# Compute LF by chain length assuming the first layer is full with 2q-gates
lf_sets, lf_qubits = two_disjoint_layers, best_qubit_chain
full_layer = [None] * (len(lf_sets[0]) + len(lf_sets[1]))
full_layer[::2] = lf_sets[0]
full_layer[1::2] = lf_sets[1]
full_layer = [(lf_qubits[0],)] + full_layer + [(lf_qubits[-1],)]
len(full_layer)

In [ ]:
assert(len(full_layer)== len(lf_qubits) + 1)

In [ ]:
pfs = [pfdf.loc[pfdf[pfdf.qubits==qubits].index[0], 'value'] for qubits in full_layer]
pfs = list(map(lambda x: x.n if x != 0 else 0, pfs))
pfs[0] = pfs[0]**2
pfs[-1] = pfs[-1]**2
np.array(pfs)

In [ ]:
len(pfs)

In [ ]:
# Approximate 1Q RB fidelities at both ends by the square root of 2Q RB fidelity at both ends.
# For example, if we have [(0, 1), (1, 2), (2, 3), (3, 4)] 2Q RB fidelities and if we want to compute a layer fidelity for [1, 2, 3],
# we approximate the 1Q filedities for (1,) and (3,) by the square root of 2Q fidelities of (0, 1) and (3, 4).
chain_lens = list(range(4, len(pfs), 2))
chain_fids = []
for length in chain_lens:
    w = length + 1  # window size
    fid_w = max(np.sqrt(pfs[s]) * np.prod(pfs[s+1:s+w-1]) * np.sqrt(pfs[s+w-1]) for s in range(len(pfs)-w+1))
    chain_fids.append(fid_w)

In [ ]:
# Plot LF by chain length
plt.title(f"Backend: {backend.name}, {RUN_TIME.strftime('%Y/%m/%d %H:%M')}")
plt.plot(
    chain_lens,
    chain_fids,
    marker="o",
    linestyle="-",
)
plt.xlim(0, chain_lens[-1]*1.05)
plt.ylim(0.95*min(chain_fids), 1)
plt.ylabel("Layer Fidelity")
plt.xlabel("Chain Length")
plt.grid()
plt.show()

In [ ]:
# Plot EPLG by chain length
num_2q_gates = [length -1 for length in chain_lens]
chain_eplgs = [1 - (fid ** (1 / num_2q)) for num_2q, fid in zip(num_2q_gates, chain_fids)]
plt.title(f"Backend: {backend.name}, {RUN_TIME.strftime('%Y/%m/%d %H:%M')}")
plt.plot(
    chain_lens,
    chain_eplgs,
    marker="o",
    linestyle="-",
)
plt.xlim(0, chain_lens[-1]*1.05)
plt.ylabel("Error per Layered Gates")
plt.xlabel("Chain Length")
plt.grid()
plt.show()